In [ ]:
from google.colab import auth, files
from google.cloud import bigquery
from datetime import datetime
import pytz

# Set up the BigQuery client
auth.authenticate_user()
client = bigquery.Client(project='bq-rf5039')

# Function to get the latest table name in a dataset
def get_latest_table_name(client, dataset_id):
    query = f"""
        SELECT table_name
        FROM `{dataset_id}.INFORMATION_SCHEMA.TABLES`
        WHERE table_type = 'BASE TABLE'
        ORDER BY creation_time DESC
        LIMIT 1
    """
    query_job = client.query(query)
    results = query_job.result()
    for row in results:
        return row.table_name
    return None

# Get the latest tables from specific datasets
latest_melted_hv_prices_table = get_latest_table_name(client, 'bq-rf5039.MELTED_HV_PRICES')
latest_addressability_table = get_latest_table_name(client, 'bq-rf5039.ADDRESSABILITY')

print(f"Latest table from MELTED_HV_PRICES: {latest_melted_hv_prices_table}")
print(f"Latest table from ADDRESSABILITY: {latest_addressability_table}")

if latest_melted_hv_prices_table and latest_addressability_table:
    # Generate a unique table name based on the current time
    paris_tz = pytz.timezone('Europe/Paris')
    current_time = datetime.now(paris_tz).strftime("%Y%m%d_%H%M%S")
    new_table_name = f'marketshare_{current_time}'

    # Create the SQL query
    query = f"""
        CREATE TABLE `bq-rf5039.MARKETSHARE.{new_table_name}` AS
        SELECT
            m.pihs_vl_status,
            m.pihs_vl_ps_status,
            m.pihs_vl_ps_sd_status,
            m.pihs_vl_ps_sd_obc_status,
            m.powertrain_id,
            m.year,
            m.product_voltage_type,
            m.product_subtype,
            m.product_type,
            m.product_level,
            m.product_power,
            m.product_supplier,
            -- Add supplier_sales_group and addressability columns only when product_supplier is not null
            CASE
              WHEN m.product_supplier IS NOT NULL THEN a.supplier_sales_group
              ELSE NULL
            END AS supplier_sales_group,
            CASE
              WHEN m.product_supplier IS NOT NULL THEN a.addressability
              ELSE NULL
            END AS addressability,
            m.volume,
            m.product_price,
            m.product_sales,
            m.lv_hv,
            m.sales_group,
            m.vehicle_design_parent,
            m.solution_owner_design_parent,
            m.vehicle_platform,
            m.global_nameplate,
            m.vehicle_program,
            m.electrification,
            m.propulsion_system_subdesign,
            m.propulsion_system_subdesign_architecture,
            m.ap_px_definition,
            m.system_voltage_v,
            m.ap_system_power_kw,
            m.ap_system_torque_nm,
            m.electric_motor_power_kw,
            m.electric_motor_torque_nm,
            m.vehicle_sub_region,
            m.drive_type,
            m.engine_eop,
            m.engine_fuel_type,
            m.layout,
            m.engine_manufacturer,
            m.engine_platform,
            m.transmission_design,
            m.transmission_sub_design,
            m.transmission_forward_speed,
            m.transmission_manufacturer,
            m.transmission_program,
            m.t_torque_nm,
            m.t_design_2,
            m.t_sub_design_2,
            m.t_forward_speeds_2,
            m.t_program_2,
            m.country,
            m.vehicle_eop_end_of_production,
            m.global_sales_segment,
            m.global_sales_sub_segment,
            m.gvw_class,
            m.vehicle_manufacturer,
            m.mnemonic_vehicle_id,
            m.production_brand,
            m.vehicle_production_plant,
            m.production_type,
            m.vehicle_ihs_region,
            m.sales_parent,
            m.vehicle_sop_start_of_production,
            m.global_make,
            m.transmission_design,
            m.battery_type,
            m.ap_battery_capacity_kwh,
            m.vehicle_id,
            m.model_code,
            m.creation_date_calendar_year,
            m.primary_eaxle_motor_technology,
            m.secondary_eaxle_motor_technology,
            m.primary_eaxle_layout,
            m.secondary_eaxle_layout,
            m.switching_techno_inverter_1,
            m.switching_techno_inverter_2,
            m.switching_techno_inverter_3,
            m.switching_techno_inverter_4,
            m.x_in_1_edrive_nb_of_functions,
            m.x_in_1_edrive_functions_description,
            m.x_in_1_edrive_emot,
            m.x_in_1_edrive_inv,
            m.x_in_1_edrive_red,
            m.x_in_1_edrive_obc,
            m.x_in_1_edrive_dcdc,
            m.x_in_1_edrive_pdu,
            m.x_in_1_edrive_vcu,
            m.x_in_1_edrive_bms,
            m.x_in_1_edrive_edc,
            m.x_in_1_edrive_ptc,
            m.x_in_1_edrive_dcac,
            m.active_parts_emotor_1,
            m.active_parts_emotor_2,
            m.active_parts_emotor_3,
            m.active_parts_emotor_4,
            m.active_parts_inverter_1,
            m.active_parts_inverter_2
        FROM
          `bq-rf5039.MELTED_HV_PRICES.{latest_melted_hv_prices_table}` m
        LEFT JOIN (
            SELECT *
            FROM `bq-rf5039.ADDRESSABILITY.{latest_addressability_table}`
            WHERE change_tracker != 'removed'
        ) a ON m.product_supplier = a.product_supplier
    """

    # Execute the query
    query_job = client.query(query)
    query_job.result()  # Wait for the query to finish

    print(f"Table created: `bq-rf5039.MARKETSHARE.{new_table_name}`")
else:
    print("Latest tables not found in MELTED_HV_PRICES or ADDRESSABILITY datasets.")
